In [1]:
from lale.datasets.openml import fetch

In [2]:
(X_train, y_train), (X_test, y_test) = fetch("jungle_chess_2pcs_raw_endgame_complete", "classification")

In [3]:
import ray
ray.shutdown()

In [4]:
# from lale.helpers import cross_val_score

In [5]:
# from lale.lib.sklearn import PCA, Nystroem, SelectKBest, RandomForestClassifier
# from lale.lib.lale import ConcatFeatures

# pipeline = (PCA() & Nystroem() & SelectKBest(k=3)) >> ConcatFeatures() >> RandomForestClassifier(n_estimators=200)
# # pipeline.visualize()

In [6]:
# sk_pipeline = pipeline.export_to_sklearn_pipeline()

In [7]:
# sk_pipeline

In [8]:
import ray
ray.init()

2021-05-11 11:55:04,828	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '9.160.6.207',
 'raylet_ip_address': '9.160.6.207',
 'redis_address': '9.160.6.207:6379',
 'object_store_address': '/tmp/ray/session_2021-05-11_11-55-04_333842_74749/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-05-11_11-55-04_333842_74749/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-05-11_11-55-04_333842_74749',
 'metrics_export_port': 52557,
 'node_id': 'bb40cc6772d56a613e2e4afb22c4d196ce239735'}

In [9]:
from com.ibm.research.ray.graph.Datamodel import Xy
from com.ibm.research.ray.graph.Datamodel import XYRef
import com.ibm.research.ray.graph.Datamodel as dm

In [21]:
from sklearn.model_selection import KFold, StratifiedKFold

In [24]:
kf = KFold(n_splits=10)

In [25]:
XY_ref_list = []
for train_index, test_index in kf.split(X_train):
    Xk_train, Xk_test = X_train[train_index], X_train[test_index]
    yk_train, yk_test = y_train[train_index], y_train[test_index]
    
    X_ref = ray.put(Xk_train)
    y_ref = ray.put(yk_train)
    Xy_ref = XYRef(X_ref, y_ref)
    XY_ref_list.append(Xy_ref)

XY_ref_ptr = ray.put(XY_ref_list)

In [26]:
pipeline = dm.Pipeline()

In [27]:
from sklearn.pipeline import FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.kernel_approximation import Nystroem
from sklearn.feature_selection import SelectKBest

In [28]:
feature_union = FeatureUnion(transformer_list=[('PCA', PCA()),
                                                ('Nystroem',
                                                 Nystroem()),
                                                ('SelectKBest',
                                                 SelectKBest(k=3))])

In [29]:
random_forest = RandomForestClassifier(n_estimators=200)

In [30]:
node_fu = dm.OrNode('feature_union', feature_union)
node_rf = dm.OrNode('randomforest', random_forest)

In [31]:
pipeline.add_edge(node_fu, node_rf)

In [32]:
import com.ibm.research.ray.graph.Runtime as rt
from com.ibm.research.ray.graph.Runtime import ExecutionType

In [33]:
import time
start = time.time()


in_args={node_fu: [XY_ref_ptr]}

out_args = rt.execute_pipeline(pipeline, ExecutionType.SCORE, in_args)

node_rf_out_args = ray.get(out_args[node_rf])
end = time.time()
print ('Time taken: ' + str(end - start))

Time taken: 75.59795594215393


In [39]:
for out_xy_ref in node_rf_out_args:
    score = ray.get(out_xy_ref[0].get_Xref())
    print(score)

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


(pid=raylet) F0511 16:57:35.978291 74765 304709120 service_based_gcs_client.cc:207] Couldn't reconnect to GCS server. The last attempted GCS server address was 9.160.6.207:50161
(pid=raylet) *** Check failure stack trace: ***
(pid=raylet)     @        0x10afa7ad2  google::LogMessage::~LogMessage()
(pid=raylet)     @        0x10af373a5  ray::RayLog::~RayLog()
(pid=raylet)     @        0x10ac2775a  ray::gcs::ServiceBasedGcsClient::ReconnectGcsServer()
(pid=raylet)     @        0x10abdfd8f  _ZZN3ray3rpc12GcsRpcClient15ReportHeartbeatERKNS0_22ReportHeartbeatRequestERKNSt3__18functionIFvRKNS_6StatusERKNS0_20ReportHeartbeatReplyEEEEENKUlS9_SC_E_clES9_SC_
(pid=raylet)     @        0x10abdf70b  ray::rpc::ClientCallImpl<>::OnReplyReceived()
(pid=raylet)     @        0x10ac28b75  _ZN5boost4asio19asio_handler_invokeIZN3ray3rpc17ClientCallManager29PollEventsFromCompletionQueueEiEUlvE_EEvRT_z
(pid=raylet)     @        0x10ac28b24  _ZN5boost4asio6detail18completion_handlerIZN3ray3rpc17ClientCallMana

In [7]:
from sklearn.model_selection import StratifiedKFold

In [8]:
cv = StratifiedKFold(5)

In [10]:
cv.split(X_train, y_train)

<generator object _BaseKFold.split at 0x7fe220bfdb50>

In [6]:
%%time 
cross_val_score(pipeline, X_train, y_train, cv=10)

CPU times: user 7min 45s, sys: 12.4 s, total: 7min 57s
Wall time: 7min 30s


[0.8168498168498168,
 0.8215118215118216,
 0.8138528138528138,
 0.8185148185148186,
 0.8178488178488178,
 0.8215118215118216,
 0.8121878121878122,
 0.8191808191808192,
 0.8021319120586275,
 0.8074616922051966]